In [207]:
import csv
import json
import re
from pathlib import Path

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [208]:
# 30k_metadata
data_description_path = Path(
	"/home/artur/Desktop/FA/data/NAKO_tabular/30k_metadata/NAKO-536 - 2023-07-13 - Beschreibung des Übergabedatensatzes.html"
)
metadata_path = Path(
	"/home/artur/Desktop/FA/data/NAKO_tabular/30k_metadata/NAKO-536_2023-07-13_Erstuntersuchung_MRT_PSN_wie_194_Metadaten.csv"
)
data_path = Path(
	"/home/artur/Desktop/FA/data/NAKO_tabular/30k_metadata/NAKO-536_2023-07-13_Erstuntersuchung_MRT_PSN_wie_194.csv"
)

root_path = data_path.parent

In [209]:
with open(data_description_path, "r") as file:
	html_content = file.read()

soup = BeautifulSoup(html_content, "html.parser")
data = pd.read_csv(data_path, sep=";", encoding="latin1", quoting=csv.QUOTE_NONE)
data.head()

,ID,basis_age,basis_sex,basis_uort,basis_status_mrt,BD17,BD01,BD02,BD035,BD14,...,ws_lk_krepitation,ws_rk_krepitation,ws_lk_knauftr,ws_rk_knauftr,ws_lh_ruheschmerz,ws_lh_innenrotation,ws_lh_schmerzinrot,ws_rh_ruheschmerz,ws_rh_innenrotation,ws_rh_schmerzinrot
0,100000,55,1,81,3,31.7,134,77,78,1,...,1.0,1.0,2.0,2.0,2.0,30.0,2.0,2.0,25.0,2.0
1,100001,57,1,21,3,36.0,138,81,69,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100002,47,1,81,3,29.1,146,87,45,1,...,2.0,1.0,2.0,2.0,2.0,25.0,2.0,2.0,20.0,2.0
3,100003,60,2,81,3,30.7,111,69,73,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100004,61,1,81,3,33.2,188,97,65,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
# Iterate attributes and scrap data from HTML to get a description for the corresponding feature
attribute_description = {}

for block in data.columns:
	attribute_box = soup.find("tr", id=block)
	if attribute_box is None:
		continue

	# get the corresponding text in the attrbiute
	description = attribute_box.find_all(string=True, limit=10)[2:]
	# Use regex to filter unwanted characters (non numerical or alphabetical)
	# pattern = re.compile(r"[^\w\s]+")
	pattern = re.compile(r"[^\w\s\xa0:°]+")
	merged_data = ["".join(filter(lambda x: not pattern.search(x), sublist)) for sublist in description]
	attr_text = "".join(merged_data).strip()

	# Conntect attribute + description
	attribute_description[block] = attr_text

In [213]:
with open(root_path / "data_description.json", "w") as outfile:
	json.dump(attribute_description, outfile)